In [1]:
import cv2
import mediapipe as mp
import time
import tensorflow as tf
from PIL import Image
import numpy as np

In [2]:
cnnModel = tf.keras.models.load_model("source/model.h5")

In [3]:
counter = 0
groupInterval = 10

sentence_counter = 0
sentenceInterval = 2

delete_counter = 0
detected_letters = []

In [4]:
def insert_into_sentence(sentence, letter):
    if len(sentence)+1 <= 17:
        sentence += letter
    else:
        sentence = sentence[1:]
        sentence += letter
    return sentence

In [ ]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FPS, 30)

mpHands = mp.solutions.hands
hands = mpHands.Hands(False, 1)
mpDraw = mp.solutions.drawing_utils

previous_Time = 0
current_Time = 0

success, img = cap.read()
height, width, channels = img.shape

letter = ""
sentence = ""
confidence = 0
sumConf = None

while True:
    success, img = cap.read()
    image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = hands.process(image)

    hand_landmarks = result.multi_hand_landmarks
    if hand_landmarks:
        for handLms in hand_landmarks:
            x_min = width
            y_min = height
            x_max = 0
            y_max = 0

            for lm in handLms.landmark:
                x, y = int(lm.x*width), int(lm.y*height)
                if x < x_min and x >= 20:
                    x_min = x
                if x > x_max and x <= width-20:
                    x_max = x
                if y < y_min and y >= 20:
                    y_min = y
                if y > y_max and y <= height-20:
                    y_max = y
            
            widthCrop = x_max - x_min
            heightCrop = y_max - y_min
            
            size = max(widthCrop, heightCrop)

#            the following code would print a rectangle around the recognized hand
            img = cv2.rectangle(img, (x_min-20, y_min-20), (x_min + size + 20, y_min + size + 20), (0, 255, 0), 2)
            
            cropped_img = Image.fromarray(image[y_min-20: y_min + size + 20, x_min-20:x_min + size + 20])
            scaled_img = np.asarray(cropped_img.resize((200, 200), Image.ANTIALIAS))
            
            res = cnnModel.predict(np.array([scaled_img]))
            
            for r in res:
                if sumConf is not None:
                    sumConf = sumConf + r
                else:
                    sumConf = r
                
            
        if counter == groupInterval:
            letterIndex = sumConf.argmax()
            confidence = sumConf.max() / sum(sumConf)
            if letterIndex < 26:
                letter = chr(letterIndex + 65)  
            elif letterIndex == 26:
                letter = 'delete'
            elif letterIndex == 27:
                letter = 'nothing'
            elif letterIndex == 28:
                letter = 'space'
                    
            if letterIndex != 27 and confidence > 0.8:
                print(f"Detected letter: {letter}({letterIndex}) with confidence {confidence}")
                cv2.putText(img, letter, (10,200), cv2.FONT_HERSHEY_PLAIN, 4, (255,255,255) , 5)
                if sentence_counter == sentenceInterval:
                    detected_letters += [letter]
                    print(detected_letters)
                    if letter == 'space':
                        sentence = insert_into_sentence(sentence, ' ')
                    elif letter == 'delete':
                        if delete_counter < 2:
                            sentence = sentence[:-1]
                            delete_counter += 1
                        elif delete_counter == 2 and detected_letters[-1] == 'delete' and detected_letters[-2] == 'delete':
                            sentence = ""
                            delete_counter = 0
                        else:
                            delete_counter = 0
                    else:
                        sentence = insert_into_sentence(sentence, letter)
                    print(f"{letter} added to sentence.\nCurrent sentence: {sentence}")
                    sentence_counter = 1
                else:
                    sentence_counter += 1
                    
            sumConf = None
            counter = 1

        else:
            if letter != 'nothing' and confidence > 0.8:
                cv2.putText(img, letter, (10,200), cv2.FONT_HERSHEY_PLAIN, 4, (255,255,255) , 5)
                    
            counter += 1
    
    cv2.putText(img, sentence, (10,450), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255) , 4)
    cv2.imshow("Image", img)
            
            
    if cv2.waitKey(10) == ord('q'):
        cv2.destroyAllWindows()
        detected_letters = []
        break

Detected letter: X(23) with confidence 0.8501044719729773
Detected letter: C(2) with confidence 0.8414344189462418
Detected letter: C(2) with confidence 0.9500942632342857
['C']
C added to sentence.
Current sentence: C
Detected letter: C(2) with confidence 0.9219352967626108
Detected letter: C(2) with confidence 0.9598241698706275
['C', 'C']
C added to sentence.
Current sentence: CC
Detected letter: A(0) with confidence 0.8345292935991551
Detected letter: U(20) with confidence 0.8761198721575687
['C', 'C', 'U']
U added to sentence.
Current sentence: CCU
Detected letter: C(2) with confidence 0.9619317676150959
Detected letter: C(2) with confidence 0.9872026097598945
['C', 'C', 'U', 'C']
C added to sentence.
Current sentence: CCUC
Detected letter: C(2) with confidence 0.9626088839155905
Detected letter: C(2) with confidence 0.9955079723023611
['C', 'C', 'U', 'C', 'C']
C added to sentence.
Current sentence: CCUCC
Detected letter: C(2) with confidence 0.9564166661319695
Detected letter: N(